In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!ls /content/drive/My\ Drive/Colab\ Notebooks/classifier_resnet_model.h5

In [0]:
! ls path

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [0]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [0]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [0]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [0]:
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

In [0]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [0]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

In [0]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

In [0]:
model = Sequential()

In [0]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [0]:
model.add(Dense(num_classes, activation='softmax'))

In [0]:
model.layers

In [0]:
model.layers[0].layers

In [0]:
model.layers[0].trainable = False

In [0]:
model.summary()

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [0]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

In [0]:
model.save('classifier_vgg16_model.h5')

## Part 2

In [0]:
from keras.models import load_model

In [0]:
vgg16_model = load_model('/content/drive/My Drive/Colab Notebooks/classifier_vgg16_model.h5')

In [0]:
resnet_model = load_model('/content/drive/My Drive/Colab Notebooks/classifier_resnet_model.h5')

In [0]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    shuffle=False,
    class_mode='categorical')

In [0]:
resNetLoss, resNetAccuracy = resnet_model.evaluate_generator(test_generator)
print('ResNet Loss:', resNetLoss)
print('ResNet Accuracy:', resNetAccuracy)

In [0]:
vggNetLoss, resNetAccuracy = vgg16_model.evaluate_generator(test_generator)
print('VGG Loss:', vggNetLoss)
print('VGG Accuracy:', resNetAccuracy)

## Part 3

In [0]:
resnet_predict = resnet_model.predict_generator(test_generator)

In [0]:
import numpy as np
predicted_resnet_class = np.argmax(resnet_predict,axis=1)
for i in range(0,5):
    if predicted_resnet_class[i]:
        print("Positive")
    else:
        print("Negative")

In [0]:
vgg_predict = vgg16_model.predict_generator(test_generator)

In [0]:
import numpy as np
predicted_vgg_class = np.argmax(vgg_predict,axis=1)
for i in range(0,5):
    if predicted_vgg_class[i]:
        print("Positive")
    else:
        print("Negative")